<a href="https://colab.research.google.com/github/tomasonjo/blogs/blob/master/llm/nvidia_neo4j_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade --quiet langchain-nvidia-ai-endpoints langchain-community neo4j langchain-core

In [2]:
import os
from typing import Any, Dict, List, Optional, Tuple, Type

from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from langchain_core.tools import tool
from langchain_nvidia_ai_endpoints import ChatNVIDIA

from langchain.agents import AgentExecutor
from langchain.agents.format_scratchpad import \
    format_to_openai_function_messages
from langchain.agents.output_parsers.openai_tools import \
    OpenAIToolsAgentOutputParser
from langchain.callbacks.manager import (AsyncCallbackManagerForToolRun,
                                         CallbackManagerForToolRun)
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.pydantic_v1 import BaseModel, Field
from langchain.schema import AIMessage, HumanMessage
from langchain.tools import BaseTool
from langchain.tools.render import format_tool_to_openai_function

In [3]:
os.environ["NVIDIA_API_KEY"] = "nvapi-"

In [4]:
llm = ChatNVIDIA(model="meta/llama-3.1-70b-instruct")

In [5]:
result = llm.invoke("How to use LLMs in combination with Graph Databases? Be concise!")
print(result.content)

Here's a concise guide on using Large Language Models (LLMs) with Graph Databases:

**Why combine LLMs with Graph Databases?**

* Enhance natural language querying and search capabilities
* Leverage LLMs for entity disambiguation, relation extraction, and ontological reasoning
* Uncover insights from complex relationships and patterns in graph data

**Architecture:**

1. **Data Integration**: Integrate graph data with LLMs through APIs, e.g., using Cypher (Neo4j) or SPARQL (RDF).
2. **Text Preprocessing**: Preprocess graph data into text formats (e.g., node/edge attributes) for LLM consumption.
3. **LLM Inference**: Use LLMs for tasks like entity recognition, relationship extraction, and question answering on the preprocessed text data.
4. **Post-processing**: Re-integrate LLM outputs with the graph database, e.g., creating new nodes, edges, or updating existing ones.

**Example Use Cases:**

* **Entity Disambiguation**: Use LLMs to identify entities in text data and resolve ambiguity,

In [6]:
os.environ["NEO4J_URI"] = "bolt://18.206.157.187:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "elevation-reservist-thousands"

graph = Neo4jGraph(refresh_schema=False)

In [7]:
graph.query(
    "CREATE FULLTEXT INDEX drug IF NOT EXISTS FOR (d:Drug) ON EACH [d.name];"
)
graph.query(
    "CREATE FULLTEXT INDEX manufacturer IF NOT EXISTS FOR (d:Manufacturer) ON EACH [d.manufacturerName];"
)

[]

In [8]:
def generate_full_text_query(input: str) -> str:
    """
    Generate a full-text search query for a given input string.

    This function constructs a query string suitable for a full-text search.
    It processes the input string by splitting it into words and appending a
    similarity threshold (~2) to each word, then combines them using the AND
    operator. Useful for mapping movies and people from user questions
    to database values, and allows for some misspelings.
    """
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()


candidate_query = """
CALL db.index.fulltext.queryNodes($index, $fulltextQuery, {limit: $limit})
YIELD node
RETURN coalesce(node.manufacturerName, node.name) AS candidate,
       labels(node)[0] AS label
"""


def get_candidates(input: str, type: str, limit: int = 3) -> List[Dict[str, str]]:
    """
    Retrieve a list of candidate entities from database based on the input string.

    This function queries the Neo4j database using a full-text search. It takes the
    input string, generates a full-text query, and executes this query against the
    specified index in the database. The function returns a list of candidates
    matching the query, with each candidate being a dictionary containing their name
    (or title) and label (either 'Person' or 'Movie').
    """
    ft_query = generate_full_text_query(input)
    candidates = graph.query(
        candidate_query, {"fulltextQuery": ft_query, "index": type, "limit": limit}
    )
    return candidates

In [9]:
get_candidates("voriconazol", "drug")

[{'candidate': 'VORICONAZOLE', 'label': 'Drug'}]

In [10]:
@tool
def get_side_effects(
    drug: Optional[str] = Field(
        description="disease mentioned in the question. Return None if no mentioned."
    ),
    min_age: Optional[int] = Field(
        description="Minimum age of the patient. Return None if no mentioned."
    ),
    max_age: Optional[int] = Field(
        description="Maximum age of the patient. Return None if no mentioned."
    ),
    manufacturer: Optional[str] = Field(
        description="manufacturer of the drug. Return None if no mentioned."
    ),
):
    """Useful for when you need to find common side effects."""
    params = {}
    filters = []
    side_effects_base_query = """
    MATCH (c:Case)-[:HAS_REACTION]->(r:Reaction), (c)-[:IS_PRIMARY_SUSPECT]->(d:Drug)
    """
    if drug and isinstance(drug, str):
        candidate_drugs = [el["candidate"] for el in get_candidates(drug, "drug")]
        if not candidate_drugs:
            return "The mentioned drug was not found"
        filters.append("d.name IN $drugs")
        params["drugs"] = candidate_drugs

    if min_age and isinstance(min_age, int):
        filters.append("c.age > $min_age ")
        params["min_age"] = min_age
    if max_age and isinstance(max_age, int):
        filters.append("c.age < $max_age ")
        params["max_age"] = max_age
    if manufacturer and isinstance(manufacturer, str):
        candidate_manufacturers = [
            el["candidate"] for el in get_candidates(manufacturer, "manufacturer")
        ]
        if not candidate_manufacturers:
            return "The mentioned manufacturer was not found"
        filters.append(
            "EXISTS {(c)<-[:REGISTERED]-(:Manufacturer {manufacturerName: $manufacturer})}"
        )
        params["manufacturer"] = candidate_manufacturers[0]

    if filters:
        side_effects_base_query += " WHERE "
        side_effects_base_query += " AND ".join(filters)
    side_effects_base_query += """
    RETURN d.name AS drug, r.description AS side_effect, count(*) AS count
    ORDER BY count DESC
    LIMIT 10
    """
    print(params)
    data = graph.query(side_effects_base_query, params=params)
    return data

In [11]:
get_side_effects("aspirin")

{'drugs': ['ASPIRINE', 'ASPRIN', 'ASPIRIN']}


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[]

In [12]:
tools = [get_side_effects]
llm_with_tools = llm.bind_tools(tools=tools)
response = llm_with_tools.invoke("What is the most common side effect when using volazopile?")
print(response)
print(response.tool_calls)

content='' additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-aa1f9f5bbac24be0841774daf56d10c1', 'type': 'function', 'function': {'name': 'get_side_effects', 'arguments': '{"drug": "volazopile"}'}}]} response_metadata={'role': 'assistant', 'content': None, 'tool_calls': [{'id': 'chatcmpl-tool-aa1f9f5bbac24be0841774daf56d10c1', 'type': 'function', 'function': {'name': 'get_side_effects', 'arguments': '{"drug": "volazopile"}'}}], 'token_usage': {'prompt_tokens': 315, 'total_tokens': 343, 'completion_tokens': 28}, 'finish_reason': 'tool_calls', 'model_name': 'meta/llama-3.1-70b-instruct'} id='run-2edf12f2-ab14-44ec-8688-b8e23cb298ad-0' tool_calls=[{'name': 'get_side_effects', 'args': {'drug': 'volazopile'}, 'id': 'chatcmpl-tool-aa1f9f5bbac24be0841774daf56d10c1', 'type': 'tool_call'}] role='assistant'
[{'name': 'get_side_effects', 'args': {'drug': 'volazopile'}, 'id': 'chatcmpl-tool-aa1f9f5bbac24be0841774daf56d10c1', 'type': 'tool_call'}]


In [13]:
prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [14]:
def _format_chat_history(chat_history: List[Tuple[str, str]]):
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer


agent = (
    {
        "input": lambda x: x["input"],
        "chat_history": lambda x: _format_chat_history(x["chat_history"])
        if x.get("chat_history")
        else [],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)


# Add typing for input
class AgentInput(BaseModel):
    input: str
    chat_history: List[Tuple[str, str]] = Field(
        ..., extra={"widget": {"type": "chat", "input": "input", "output": "output"}}
    )


class Output(BaseModel):
    output: Any


agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True).with_types(
    input_type=AgentInput, output_type=Output
)

In [15]:
agent_executor.invoke(
    {"input": "What are the most common side effects when using lyrica?"}
)




> Entering new AgentExecutor chain...

Invoking: `get_side_effects` with `{'drug': 'lyrica'}`


{'drugs': ['LYRICA', 'LYRICA CR']}
[{'drug': 'LYRICA', 'side_effect': 'Pain', 'count': 32}, {'drug': 'LYRICA', 'side_effect': 'Fall', 'count': 21}, {'drug': 'LYRICA', 'side_effect': 'Intentional product use issue', 'count': 20}, {'drug': 'LYRICA', 'side_effect': 'Insomnia', 'count': 19}, {'drug': 'LYRICA', 'side_effect': 'Feeling abnormal', 'count': 18}, {'drug': 'LYRICA', 'side_effect': 'Drug ineffective', 'count': 18}, {'drug': 'LYRICA', 'side_effect': 'Memory impairment', 'count': 17}, {'drug': 'LYRICA', 'side_effect': 'Withdrawal syndrome', 'count': 17}, {'drug': 'LYRICA', 'side_effect': 'Malaise', 'count': 16}, {'drug': 'LYRICA', 'side_effect': 'Intentional product misuse', 'count': 15}]Based on the data, the most common side effects of Lyrica are pain, fall, intentional product use issue, insomnia, feeling abnormal, drug ineffective, memory impairment, withdrawal syndrome, malaise, a

{'input': 'What are the most common side effects when using lyrica?',
 'output': 'Based on the data, the most common side effects of Lyrica are pain, fall, intentional product use issue, insomnia, feeling abnormal, drug ineffective, memory impairment, withdrawal syndrome, malaise, and intentional product misuse.'}

In [16]:
agent_executor.invoke(
    {
        "input": "What are the most common side effects when using lyrica for people below 35 years old?"
    }
)




> Entering new AgentExecutor chain...

Invoking: `get_side_effects` with `{'min_age': 0, 'max_age': 35, 'drug': 'lyrica'}`


{'drugs': ['LYRICA', 'LYRICA CR'], 'max_age': 35}
[{'drug': 'LYRICA', 'side_effect': 'Sinusitis', 'count': 1}, {'drug': 'LYRICA', 'side_effect': 'Hypoacusis', 'count': 1}, {'drug': 'LYRICA', 'side_effect': 'Fall', 'count': 1}, {'drug': 'LYRICA', 'side_effect': 'Brain injury', 'count': 1}, {'drug': 'LYRICA', 'side_effect': 'Amnesia', 'count': 1}, {'drug': 'LYRICA', 'side_effect': 'Off label use', 'count': 1}, {'drug': 'LYRICA', 'side_effect': 'Impaired quality of life', 'count': 1}, {'drug': 'LYRICA', 'side_effect': 'Somnolence', 'count': 1}, {'drug': 'LYRICA', 'side_effect': 'Drug ineffective for unapproved indication', 'count': 1}]Based on the provided data, the most common side effects of Lyrica for people below 35 years old are sinusitis, hypoacusis, fall, brain injury, amnesia, off-label use, impaired quality of life, somnolence, and drug ineffective for un

{'input': 'What are the most common side effects when using lyrica for people below 35 years old?',
 'output': 'Based on the provided data, the most common side effects of Lyrica for people below 35 years old are sinusitis, hypoacusis, fall, brain injury, amnesia, off-label use, impaired quality of life, somnolence, and drug ineffective for unapproved indication.'}

In [17]:
agent_executor.invoke(
    {
        "input": "What are the most common side effects when for drugs manufactured by acadia?"
    }
)




> Entering new AgentExecutor chain...

Invoking: `get_side_effects` with `{'manufacturer': 'acadia'}`


{'manufacturer': 'ACADIA PHARMACEUTICALS'}
[{'drug': 'NUPLAZID', 'side_effect': 'Hallucination', 'count': 13}, {'drug': 'NUPLAZID', 'side_effect': 'Confusional state', 'count': 7}, {'drug': 'NUPLAZID', 'side_effect': 'Fall', 'count': 6}, {'drug': 'NUPLAZID', 'side_effect': 'Delusion', 'count': 5}, {'drug': 'NUPLAZID', 'side_effect': 'Gait disturbance', 'count': 5}, {'drug': 'NUPLAZID', 'side_effect': 'Fatigue', 'count': 4}, {'drug': 'NUPLAZID', 'side_effect': 'Abnormal behaviour', 'count': 3}, {'drug': 'NUPLAZID', 'side_effect': 'Product dose omission issue', 'count': 3}, {'drug': 'NUPLAZID', 'side_effect': 'Agitation', 'count': 3}, {'drug': 'NUPLAZID', 'side_effect': 'Death', 'count': 3}]The most common side effects for drugs manufactured by Acadia are Hallucination, Confusional state, Fall, Delusion, Gait disturbance, Fatigue, Abnormal behaviour, Product dose omission issue, Agit

{'input': 'What are the most common side effects when for drugs manufactured by acadia?',
 'output': 'The most common side effects for drugs manufactured by Acadia are Hallucination, Confusional state, Fall, Delusion, Gait disturbance, Fatigue, Abnormal behaviour, Product dose omission issue, Agitation and Death.'}